In [43]:
#inserting libraries for the prject

import random #random forest generation
import numpy as np #vectorized computation
import pandas as pd #processing data to dataframes
import requests
import matplotlib.pyplot as plt #visualizing data via plotting
%matplotlib inline
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans #importing KMeans
from sklearn.datasets.samples_generator import make_blobs
import bs4 as bs
import urllib.request
import matplotlib.cm as cm
import matplotlib.colors as colors
print("All libraries imported")

All libraries imported


# Part - Project 1 : Text Scrapping with Beautiful Soup

1) Start by creating a new Notebook for this assignment
2) Scrape the following Wikipedia page in order to obtain the data
3) Create the dataframe according to the bullets

#### Obtaining data from the following Wikipedia page

In [44]:
#The dataframe must contain three columns: Postalcode, Borough, Neighborhood

source = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read()
soup = bs.BeautifulSoup(source,"html.parser")

table = soup.find("table")
table_rows = table.find_all("tr")

l = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)

In [45]:
df = pd.DataFrame(l, columns=["Postal Code","Borough","Neighborhood"])
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


#### Removing the boroughs with "Not assigned" label

In [46]:
#Process the cells that have assigned boroughs. Ignore the remaining boroughs with "Not assigned"
df = df[df.Borough != "Not assigned"]
df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [47]:
#Gather all neighborhouds in each one borough
df = df.groupby(["Postal Code", "Borough"]).agg(", ".join)
df = df.reset_index()
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### Assigning the "not assigned" neighbourhoods with the same name as their boroughs

In [48]:
#"Not assigned" neighborhoods will be the same as their borough
df.loc[df['Neighborhood']=="Not assigned", ["Neighborhood"]] = "Queen\'s Park'"
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### Checking the shape of the dataframe

In [49]:
#Use the .shape method to print the number of rows of the dataframe
df.shape

(103, 3)

# Part - Project 2 : Finding the latitude and the longitude coordinates of each neighborhood

Geocoder was not working, so I used the link to download the csv file

In [50]:
#I download the .csv file from the link http://cocl.us/Geospatial_data
df_coord = pd.read_csv("https://cocl.us/Geospatial_data")
Data_Location = pd.merge(df, df_coord, on = "Postal Code")

#### Checking the data

In [51]:
Data_Location.head(10)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### Creating a new dataframe which contains only the borough "North York"

In [52]:
Toronto_data = Data_Location[Data_Location["Borough"] == "Downtown Toronto"].reset_index(drop=True)
Toronto_data.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


# Part - Project 3 : Exploring Toronto map

### First task is to generate Toronto Map and plotting North York neigborhoods

In [53]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

TorLat = 43.651070 #This is the Latitude
TorLong = -79.347015 #This is the Longitude
d_toronto = folium.Map(location=[TorLat,TorLong], zoom_start=10) #call Folium to generate the map
for lat, lng, borough, neighbourhood in zip(Toronto_data["Latitude"], Toronto_data["Longitude"], Toronto_data["Borough"], Toronto_data["Neighborhood"]):
    label = "{}, {}".format(Toronto_data,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color="blue",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.6,
        parse_html=False).add_to(d_toronto)

d_toronto

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / ^C
failed with initial frozen solve. Retrying with flexible solve.

CondaError: KeyboardInterrupt



ModuleNotFoundError: No module named 'folium'

## Inserting information from Foursquare

In [ ]:
CLIENT_ID="SP3M1HLBSEY5PZKQFPOL20ZIA2DNJKVXLQXAHYECLYAV3UXZ"
CLIENT_SECRET = "Z2WROULQXZBHDQT4Y0HB5COC4OAEZSWSWO2YNGMN3PPDO3FZ"
VERSION = "20180604"
LIMIT = 100

### Creating a function which explore the nighborhood in downtown Toronto

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        #create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        venues_list.append([(
            name,
            lat,
            lng,
            v["venue"]["name"],
            v["venue"]["location"]["lat"],
            v["venue"]["location"]["lng"],
            v["venue"]["categories"][0]["name"]) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Neighborhood",
                    "Neighborhood Latitude",
                    "Neighborhood Longitude",
                    "Venue",
                    "Venue Latitude",
                    "Venue Longitude",
                    "Venue Category"]
    return(nearby_venues)

## Get all nearby venues in neighborhoods

In [ ]:
dToronto_venues = getNearbyVenues(names=Toronto_data["Neighborhood"],
                                    latitudes=Toronto_data["Latitude"],
                                    longitudes=Toronto_data["Longitude"]
                                   )

### Getting Informaction about venues

In [ ]:
print("There are a total of", dToronto_venues.shape[0], "venues in downtown Toronto grouped in", len(dToronto_venues["Venue Category"].unique()),"categories")
dToronto_venues.head()

## Further Analysis of downtown Toronto neighbourhoods

In [ ]:
dToronto_spots = pd.get_dummies(dToronto_venues[["Venue Category"]], prefix="", prefix_sep="")

#Then, adding neighbourhood coliumn to dataframe back
dToronto_spots["Neighborhood"] = dToronto_venues["Neighborhood"]

#transfer neighbourhood column to the first one
fixed_columns = [dToronto_spots.columns[-1]] + list(dToronto_spots.columns[:-1])
dToronto_spots = dToronto_spots[fixed_columns]
Group_neighbourhood = dToronto_spots.groupby('Neighborhood').mean().reset_index()

### Top 3 venue's categories per neighborhood

In [ ]:
venues = 4
print("***** These are the top 3 venue categories in each neighborhood of downtown Toronto *****")
print("\n")
for hood in Group_neighbourhood["Neighborhood"]:
    print("----"+hood+"----")
    tbl = Group_neighbourhood[Group_neighbourhood["Neighborhood"]==hood].T.reset_index()
    tbl.columns = ["venue", "freq"]
    tbl = tbl.iloc[1:]
    tbl["freq"] = tbl["freq"].astype(float)
    tbl = tbl.round({"freq":2})
    print(tbl.sort_values("freq", ascending=False).reset_index(drop=True).head(venues))
    print("\n")

### The 5 most common venues per neighborhood

#### Creating a function with the 5 most common venues

In [ ]:
def return_5most_venues(row,n):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:n]

#### Apply the factor 8

In [ ]:
venues = 8

indicators = ["st", "nd", "rd"]

#below we create the columns according to the number of top venues
columns = ["Neighborhood"]
for ind in np.arange(venues):
    try:
        columns.append("{}{} Most common venue".format(ind+1, indicators[ind]))
    except:
         columns.append("{}th Most common venue".format(ind+1))
            
#next we create a new dataframe
neighborhoods_venues_sort = pd.DataFrame(columns=columns)
neighborhoods_venues_sort["Neighborhood"] = Group_neighbourhood["Neighborhood"]

for ind in np.arange(Group_neighbourhood.shape[0]):
    neighborhoods_venues_sort.iloc[ind, 1:] = return_5most_venues(Group_neighbourhood.iloc[ind, :], venues)
    
neighborhoods_venues_sort.head()

### Clustering the downtown Toronto Neighborhood

In [ ]:
#set the number of the clusters
kclusters = 5

dToronto_clusters = Group_neighbourhood.drop("Neighborhood", 1)

#applying the kmeans clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dToronto_clusters)

kmeans.labels_[0:8]

In [ ]:
#adding the labels of clustering
neighborhoods_venues_sort.insert(0, "Cluster Labels", kmeans.labels_)
dToronto_clustered = Toronto_data


dToronto_clustered = dToronto_clustered.join(neighborhoods_venues_sort.set_index("Neighborhood"), on="Neighborhood")
dToronto_clustered.head()

## Visualizing clusters on the map

In [ ]:
toronto_cluster_map =folium.Map(location=[TorLat,TorLong], zoom_start=14)

x =np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
dToronto_clustered['Cluster Labels']
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dToronto_clustered['Latitude'], dToronto_clustered['Longitude'], dToronto_clustered['Neighborhood'], dToronto_clustered['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.6).add_to(toronto_cluster_map)
       
toronto_cluster_map

## Exploring each cluster

In [ ]:
Cluster0 = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == 0,
                                  dToronto_clustered.columns[[1] + list(range(6, dToronto_clustered.shape[1]))]]
Cluster0

In [ ]:
Cluster1 = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == 1,
                                  dToronto_clustered.columns[[1] + list(range(6, dToronto_clustered.shape[1]))]]
Cluster1

In [ ]:
Cluster2 = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == 2,
                                  dToronto_clustered.columns[[1] + list(range(6, dToronto_clustered.shape[1]))]]
Cluster2

In [ ]:
Cluster3 = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == 3,
                                  dToronto_clustered.columns[[1] + list(range(6, dToronto_clustered.shape[1]))]]
Cluster3

In [ ]:
Cluster4 = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == 4,
                                  dToronto_clustered.columns[[1] + list(range(6, dToronto_clustered.shape[1]))]]
Cluster4

## Naming clusters

In [ ]:
Cluster_Name = pd.DataFrame(columns = ['Cluster num', 'Cluster Labels'])
Cluster_Name['Cluster num'] = dToronto_clustered['Cluster Labels']
test = []
clusName = []
for j in range(5):
    t = []
    Clus = dToronto_clustered.loc[dToronto_clustered['Cluster Labels'] == j, dToronto_clustered.columns[[1] + 
list(range(6, dToronto_clustered.shape[1]))]]
    for i in range(1,5):
        t.append(Clus.loc[0:,Cluster0.columns[i]].mode()[0]) 
    clusName = list(set(t[1:]))
    #Cluster_Name.loc(dToronto_clustered['Cluster Labels'],'Cluster name') = str(clusNames)
    print(clusName)
    test.append(clusName)

for k in range(5):
    Cluster_Name.loc[dToronto_clustered['Cluster Labels'] == k, 'Cluster Labels'] = ", ".join(test[k])

In [ ]:
Cluster_Name


## Visualizing the map with new named clusters

In [ ]:
new_map_clusters = folium.Map(location=[TorLat, TorLong], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
dToronto_clustered['Cluster Labels']
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, k in zip(dToronto_clustered['Latitude'], dToronto_clustered['Longitude'], dToronto_clustered['Neighborhood'], Cluster_Name['Cluster Labels'], Cluster_Name['Cluster num']):
    label = folium.Popup('Neighborhood: ' + str(poi) + '. Venues: ' + cluster, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[k-1],
        fill=True,
        fill_color=rainbow[k-1],
        fill_opacity=0.6).add_to(new_map_clusters)
       
new_map_clusters